In [89]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [90]:
data = pd.read_csv('./data/PoPCites2.csv')
#data = pd.read_csv('data/PoPCites2.csv',usecols = '')

# RQ1: Which technologies have been investigated in the last decade?

# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

# RQ3: How stable is the community working on the topics (new authors emerging, authors staying on for the whole time,
# or authors “leaving” the area)?

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Cites           200 non-null    int64  
 1   Authors         200 non-null    object 
 2   Title           200 non-null    object 
 3   Year            200 non-null    int64  
 4   Source          183 non-null    object 
 5   Publisher       200 non-null    object 
 6   ArticleURL      200 non-null    object 
 7   CitesURL        200 non-null    object 
 8   GSRank          200 non-null    int64  
 9   QueryDate       200 non-null    object 
 10  Type            37 non-null     object 
 11  DOI             0 non-null      float64
 12  ISSN            0 non-null      float64
 13  CitationURL     0 non-null      float64
 14  Volume          0 non-null      float64
 15  Issue           0 non-null      float64
 16  StartPage       0 non-null      float64
 17  EndPage         0 non-null      flo

In [91]:
#Funktionen
def Count(data,count):
    cnt = Counter()
    for item in data: 
        cnt[item] += 1
    return cnt.most_common(count)

def CountCites(data,count):
    cnt = Counter()
    for item in data: 
        cnt[item] += 1
    return cnt.most_common(count)

def CountWords(data,count):
    cnt = Counter()
    #Pandas.Series durchlaufen um die Title zu bekommen
    for index, value in data.items():
        words = value.lower().split(' ')
        #Title in Wörter teilen
        for word in words:
            #Aussortieren der Stopwords
            if not word in stopwords:
                #Für bestimmte Begriffe
                #for word in wordlist:
                cnt[word] += 1
    return cnt.most_common(count)

def CountToTfidf(counts):
    transformer = TfidfTransformer(smooth_idf=False)
    
    #Gewichtet die CountMatrix
    tfidf = transformer.fit_transform(counts)

    #Gibt Array zurück
    return tfidf.toarray()


def ListToDataArray(l,year):
    ld = pd.DataFrame.from_dict(l)
    ld = ld.rename(columns={ 0:'word',1:year})
    ld = ld.set_index('word')
    #Gibt Pandas.DataFrame zurück
    return ld


def TfidfVector(data):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data)
    print(vectorizer.get_feature_names())
    print(X.toarray())
    
def CountVector(data):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(data)
    print(vectorizer.get_feature_names())
    return X.toarray()
    
def TitleFilteredByYear(data,year):
    databyyear = data['Title'].loc[data['Year'] == year]
    return databyyear

def CountWordsByYear(data,year):
    #d = Pandas.Series
    d = TitleFilteredByYear(data,year)
    l = CountWords(d,30)
    print(type(l))
   # r = []
    #for x in word,count:
     #   if(type(x) != NoneType):
      #      r.append(x)
    #Gibt List zurück
    return l
    
    
    
    


In [92]:
#Variablen erstellen
title = data['Title']
publisher = data['Publisher']
year = data['Year']

wordlist = [
    'DevOps',
    'engineering',
    'Software',
    'Engineering',
    'software'
    
]
stopwords =[
    'this',
    'is ',
    'none',
    'in',
    'of',
    'A',
    'a',
    'and',
    'for',
    'with',
    'on',
    'the',
    'to',
    'an'
]


In [186]:
def MergeDataFrames(data,anzahlJahre):
    year = datetime.datetime.now()
    year = year.year
    da = pd.DataFrame()
    for y in range(year - anzahlJahre, year):
        l = CountWordsByYear(data,y)  
        p = ListToDataArray(l,y)
        if(da.empty):
            da = p
        else:
            da = da.merge(p, on= 'word' ,how='inner')
    return da        

In [94]:
l1 = CountWordsByYear(data,2017)

l1


<class 'list'>


[('software', 65),
 ('engineering', 56),
 ('engineering:', 7),
 ('teaching', 6),
 ('analysis', 6),
 ('systematic', 6),
 ('research', 5),
 ('course', 5),
 ('from', 4),
 ('study', 4),
 ('data', 4),
 ('methods', 3),
 ('empirical', 3),
 ('challenges', 3),
 ('industry', 3),
 ('experience', 3),
 ('applying', 3),
 ('using', 3),
 ('education', 3),
 ('mapping', 3),
 ('agile', 3),
 ('perspective', 3),
 ('guidelines', 3),
 ('de', 3),
 ('based', 3),
 ('model-driven', 2),
 ('use', 2),
 ('crowdsourcing', 2),
 ('continuous', 2),
 ('directions', 2)]

In [95]:
l2 = CountWordsByYear(data,2018)
l2

<class 'list'>


[('software', 79),
 ('engineering', 61),
 ('research', 10),
 ('engineering:', 10),
 ('learning', 6),
 ('using', 6),
 ('systematic', 5),
 ('study', 5),
 ('search-based', 5),
 ('from', 5),
 ('data', 5),
 ('continuous', 5),
 ('empirical', 4),
 ('based', 4),
 ('mapping', 4),
 ('knowledge', 4),
 ('project', 4),
 ('tools', 3),
 ('education', 3),
 ('measuring', 3),
 ('experience', 3),
 ('approaches', 3),
 ('towards', 3),
 ('model', 3),
 ('practice', 3),
 ('students', 3),
 ('what', 3),
 ('role', 3),
 ('value', 3),
 ('systems', 3)]

In [96]:
l3 = CountWordsByYear(data,2019)
l3

<class 'list'>


[('software', 48),
 ('engineering', 38),
 ('study', 11),
 ('systematic', 9),
 ('mapping', 7),
 ('engineering:', 5),
 ('literature', 4),
 ('design', 4),
 ('case', 3),
 ('analysis', 3),
 ('systems', 3),
 ('studies', 3),
 ('education', 3),
 ('practices', 3),
 ('empirical', 3),
 ('from', 3),
 ('end-user', 3),
 ('machine', 2),
 ('industrial', 2),
 ('sethesaurus:', 2),
 ('wordnet', 2),
 ('progression', 2),
 ('model', 2),
 ('goals,', 2),
 ('challenges,', 2),
 ('start-ups', 2),
 ('reviews', 2),
 ('cerse-catalog', 2),
 ('research', 2),
 ('safety', 2)]

In [184]:
MergeDataFrames(data

<class 'list'>
<class 'list'>
<class 'list'>


,2018,2019,2020
word,,,
software,79,48,17
engineering,61,38,15
research,10,2,2
systematic,5,9,3


In [98]:
# Load English tokenizer, tagger, parser and NER
#nlp = spacy.load("en_core_web_sm")

# Process whole documents
#doc = nlp(text)

# Analyze syntax
#print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
#print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
#for entity in doc.ents:
##print(entity.text, entity.label_)

In [ ]:

#Dokumentenähnlichkeit x Graphanalyse

#Wachstum darstellen


In [ ]:
#data2018 = TitleFilteredByYear(data,2018)
#data['Cites'].plot.bar()

In [ ]:
#TfidfVector(data)


In [ ]:
#data2018